<a href="https://colab.research.google.com/github/juanmanuelvillacis/lab_recommender_system/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [5]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Iron Hack/LABs/csv_docs/customer_product_sales.csv')

In [7]:
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [51]:
df_product = data.groupby(['CustomerID','ProductName']).sum('Quantity').reset_index()
df_product

,CustomerID,ProductName,SalesID,ProductID,Quantity
0,33,Apricots - Dried,172592,324,1
1,33,Assorted Desserts,5363705,219,1
2,33,Bandage - Flexible Neon,6299165,273,1
3,33,"Bar Mix - Pina Colada, 355 Ml",3147536,281,1
4,33,"Beans - Kidney, Canned",6507033,300,1
...,...,...,...,...,...
63623,98200,Vol Au Vents,5160346,272,50
63624,98200,Wasabi Powder,4438427,80,25
63625,98200,Wine - Fume Blanc Fetzer,6226060,76,25
63626,98200,Wine - Hardys Bankside Shiraz,266676,360,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [52]:
pivot = df_product.pivot_table(values='Quantity',index='CustomerID',columns='ProductName')
pivot.fillna(0,inplace=True)
pivot

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,Bacardi Breezer - Tropical,Bagel - Plain,Baking Powder,Banana - Leaves,Banana Turning,Bananas,Bandage - Fexible 1x3,Bandage - Flexible Neon,Bar - Granola Trail Mix Fruit Nut,"Bar Mix - Pina Colada, 355 Ml",Barramundi,Bay Leaf,Beans - Kidney White,"Beans - Kidney, Canned","Beans - Kidney, Red Dry",Beans - Wax,"Beef - Chuck, Boneless",Beef - Ground Medium,"Beef - Ground, Extra Lean, Fresh",Beef - Inside Round,Beef - Montreal Smoked Brisket,Beef - Prime Rib Aaa,Beef - Rib Eye Aaa,Beef - Short Loin,Beef - Striploin Aa,"Beef - Tenderlion, Center Cut",Beef - Texas Style Burger,Beef - Top Sirloin,Beef - Top Sirloin - Aaa,Beef Ground Medium,...,Whmis - Spray Bottle Trigger,Wiberg Super Cure,Wine - Alsace Gewurztraminer,Wine - Blue Nun Qualitatswein,"Wine - Cahors Ac 2000, Clos",Wine - Chablis 2003 Champs,Wine - Charddonnay Errazuriz,Wine - Chardonnay South,Wine - Crozes Hermitage E.,Wine - Ej Gallo Sierra Valley,Wine - Fume Blanc Fetzer,Wine - Gato Negro Cabernet,Wine - Hardys Bankside Shiraz,Wine - Magnotta - Belpaese,Wine - Magnotta - Cab Sauv,"Wine - Magnotta, Merlot Sr Vqa",Wine - Pinot Noir Latour,Wine - Prosecco Valdobiaddene,"Wine - Red, Colio Cabernet","Wine - Red, Cooking","Wine - Red, Harrow Estates, Cab",Wine - Redchard Merritt,Wine - Ruffino Chianti,Wine - Sogrape Mateus Rose,Wine - Toasted Head,Wine - Two Oceans Cabernet,Wine - Valpolicella Masi,Wine - Vidal Icewine Magnotta,Wine - Vineland Estate Semi - Dry,Wine - White Cab Sauv.on,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [11]:
pdist(pivot, metric='euclidean')

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [54]:
sim_matrix = pd.DataFrame(1/(1 + squareform(pdist(pivot, metric='euclidean'))),index=pivot.index,columns=pivot.index)
sim_matrix


CustomerID,33,200,264,356,412,464,477,639,649,669,694,756,883,891,1008,1034,1066,1072,1336,1428,1435,1534,1577,1594,1754,1839,1920,2187,2329,2503,2556,2566,2582,2617,2686,2754,2776,2902,2915,2939,...,94438,94547,94599,94910,94951,95017,95034,95059,95078,95121,95314,95372,95819,96024,96088,96272,96522,96524,96560,96615,96666,96684,97029,97052,97063,97093,97201,97282,97324,97495,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,0.078977,0.083651,0.078977,0.076678,0.083972,0.080920,0.079792,0.085297,0.081800,0.075956,0.076923,0.083020,0.084959,0.078187,0.082100,0.076678,0.081503,0.082100,0.077674,0.085983,0.081800,0.077171,0.076923,0.083333,0.077929,0.075956,0.082403,0.078448,0.077421,0.080350,...,0.004440,0.005273,0.004894,0.004704,0.004348,0.004077,0.005222,0.004529,0.004493,0.004322,0.004431,0.004687,0.004476,0.004551,0.004248,0.004450,0.004948,0.004205,0.004215,0.004531,0.004943,0.004435,0.004802,0.004655,0.004471,0.004975,0.004779,0.004478,0.004032,0.004782,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,0.075025,0.075956,0.076435,0.073478,0.080350,0.074798,0.077171,0.083972,0.074573,0.072844,0.073265,0.076923,0.076923,0.075720,0.077674,0.075720,0.082100,0.076194,0.075720,0.080350,0.076435,0.071824,0.075025,0.079246,0.074573,0.073265,0.077421,0.075486,0.074129,0.078711,...,0.004456,0.005293,0.004915,0.004708,0.004363,0.004083,0.005210,0.004542,0.004497,0.004325,0.004429,0.004691,0.004475,0.004564,0.004240,0.004436,0.004974,0.004227,0.004231,0.004537,0.004918,0.004445,0.004812,0.004641,0.004473,0.004953,0.004775,0.004491,0.004041,0.004783,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,0.080070,0.082403,0.083651,0.077171,0.088152,0.081503,0.079792,0.087410,0.077929,0.076923,0.075956,0.082403,0.081210,0.081503,0.079792,0.076678,0.082709,0.079792,0.076194,0.082709,0.080070,0.076678,0.078977,0.088152,0.074129,0.078448,0.081210,0.081210,0.080070,0.080920,...,0.004448,0.005276,0.004913,0.004699,0.004348,0.004082,0.005218,0.004533,0.004493,0.004312,0.004410,0.004677,0.004467,0.004553,0.004236,0.004439,0.004972,0.004211,0.004209,0.004517,0.004916,0.004444,0.004816,0.004642,0.004474,0.004963,0.004790,0.004467,0.004045,0.004768,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,0.076923,0.076435,0.082403,0.072637,0.085297,0.077171,0.075720,0.082709,0.078448,0.075025,0.075025,0.081210,0.077421,0.074798,0.078187,0.078711,0.079792,0.079246,0.077171,0.083333,0.080634,0.073054,0.075956,0.083333,0.076923,0.074573,0.079518,0.077929,0.075956,0.078187,...,0.004435,0.005293,0.004904,0.004696,0.004345,0.004087,0.005203,0.004526,0.004490,0.004310,0.004427,0.004679,0.004473,0.004558,0.004234,0.004441,0.004956,0.004214,0.004231,0.004524,0.004913,0.004441,0.004807,0.004644,0.004473,0.004963,0.004776,0.004491,0.004042,0.004778,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,0.075956,0.078448,0.081210,0.076194,0.083333,0.074798,0.075255,0.082709,0.078977,0.074573,0.075025,0.081800,0.076923,0.076194,0.077674,0.076678,0.076678,0.078187,0.077171,0.080350,0.078977,0.073910,0.075486,0.079792,0.073693,0.071626,0.076923,0.074573,0.074129,0.078187,...,0.004454,0.005308,0.004918,0.004706,0.004345,0.004087,0.005206,0.004549,0.004499,0.004307,0.004431,0.004676,0.004471,0.004577,0.004230,0.004456,0.004978,0.004220,0.004222,0.004523,0.004918,

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [84]:
top_5 = sim_matrix.sort_values(33,ascending=False).iloc[1:6,1]
top_5_id = top_5.index.values
top_5_id

array([3317, 3535,  264, 2503, 3305])

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [57]:
df_simil = df_product[df_product['CustomerID'].isin (top_5_id)]

In [58]:
df_simil

,CustomerID,ProductName,SalesID,ProductID,Quantity
131,264,Apricots - Halves,3730301,278,1
132,264,Apricots Fresh,5035449,158,1
133,264,Bacardi Breezer - Tropical,756802,49,1
134,264,Bagel - Plain,6728381,262,1
135,264,Banana - Leaves,6298459,271,1
...,...,...,...,...,...
3003,3535,Tray - 16in Rnd Blk,6122047,166,1
3004,3535,Wanton Wrap,2643670,253,1
3005,3535,Wine - Blue Nun Qualitatswein,3314104,269,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",6563480,192,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [59]:
df_products_top5 = df_simil.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False)
df_products_top5

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [ ]:
chosen = df_product[df_product['CustomerID']==33][['ProductName','Quantity']]
chosen

In [104]:
rec_prod = chosen.merge(df_products_top5,on='ProductName',how='outer').sort_values(by='Quantity_y',ascending=False)
rec_prod = rec_prod[rec_prod['Quantity_x'].isna()][['ProductName','Quantity_y']].head()
rec_prod

,ProductName,Quantity_y
51,"Pepper - Black, Whole",3.0
52,Knife Plastic - White,3.0
53,Butter - Unsalted,3.0
54,Wine - Ej Gallo Sierra Valley,3.0
55,Wine - Blue Nun Qualitatswein,3.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [91]:
ids = data['CustomerID'].unique()
#ids = [80890,12598,7623]


In [105]:
recomendations = list()
for i,a in enumerate(ids):
  top_5 = sim_matrix[a].sort_values(ascending=False)[1:].head(5)
  df_simil = df_product[df_product['CustomerID'].isin (top_5.index)]
  df_products_top5 = df_simil.groupby('ProductName')[["ProductName","Quantity"]].sum("Quantity").sort_values(ascending=False, by="Quantity")
  #7
  chosen = df_product[df_product['CustomerID']==a][['ProductName','Quantity']]
  rec_prod = chosen.merge(df_products_top5,on='ProductName',how='outer').sort_values(by='Quantity_y',ascending=False)
  rec_prod = rec_prod[rec_prod['Quantity_x'].isna()][['ProductName','Quantity_y']].head()
  rec_prod = list(rec_prod['ProductName'])
  reco_all={'CustomerID':i,'Rec.Product1':rec_prod[0],'Rec.Product2':rec_prod[1],'Rec.Product3':rec_prod[2],
            'Rec.Product4':rec_prod[3],'Rec.Product5':rec_prod[4],}
  recomendations.append(reco_all)



##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [106]:
df_res = pd.DataFrame(recomendations)
df_res

,CustomerID,Rec.Product1,Rec.Product2,Rec.Product3,Rec.Product4,Rec.Product5
0,0,Wine - Two Oceans Cabernet,Jagermeister,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,1,Fondant - Icing,Grenadine,Mangoes,Veal - Osso Bucco,Flour - Whole Wheat
2,2,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Juice - V8 Splash,"Veal - Brisket, Provimi,bnls",Anchovy Paste - 56 G Tube
3,3,Olives - Stuffed,Wine - Charddonnay Errazuriz,Assorted Desserts,Soup - Campbells Tomato Ravioli,Juice - Orange
4,4,"Tart Shells - Sweet, 4",Artichokes - Jerusalem,Sprouts - Baby Pea Tendrils,Chocolate - Dark,"Appetizer - Mini Egg Roll, Shrimp"
...,...,...,...,...,...,...
995,995,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
996,996,"Mushroom - Porcini, Dry",Bandage - Flexible Neon,Sugar - Fine,Table Cloth 62x114 White,Hot Chocolate - Individual
997,997,Macaroons - Two Bite Choc,Apricots - Halves,"Potatoes - Instant, Mashed",Arizona - Green Tea,"Soup - Campbells, Lentil"
998,998,"Veal - Inside, Choice",Bread Fig And Almond,Campari,Fondant - Icing,"Salmon - Atlantic, Skin On"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [107]:
def make_reco (dist):
  ids = data['CustomerID'].unique()
  recomendations = list()
  for i,a in enumerate(ids):
    top_5 = sim_matrix[a].sort_values(ascending=False)[1:].head(5)
    df_simil = df_product[df_product['CustomerID'].isin (top_5.index)]
    df_products_top5 = df_simil.groupby('ProductName')[["ProductName","Quantity"]].sum("Quantity").sort_values(ascending=False, by="Quantity")
    #7
    chosen = df_product[df_product['CustomerID']==a][['ProductName','Quantity']]
    rec_prod = chosen.merge(df_products_top5,on='ProductName',how='outer').sort_values(by='Quantity_y',ascending=False)
    rec_prod = rec_prod[rec_prod['Quantity_x'].isna()][['ProductName','Quantity_y']].head()
    rec_prod = list(rec_prod['ProductName'])
    reco_all={'CustomerID':i,'Rec.Product1':rec_prod[0],'Rec.Product2':rec_prod[1],'Rec.Product3':rec_prod[2],
              'Rec.Product4':rec_prod[3],'Rec.Product5':rec_prod[4],}
    recomendations.append(reco_all)
  return pd.DataFrame(recomendations)



In [108]:
# metrica "correlation"
sim_matrix = pd.DataFrame(1/(1 + squareform(pdist(pivot, metric='correlation'))),index=pivot.index,columns=pivot.index)


make_reco(sim_matrix)

,CustomerID,Rec.Product1,Rec.Product2,Rec.Product3,Rec.Product4,Rec.Product5
0,0,Wine - Two Oceans Cabernet,Jagermeister,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,1,Fondant - Icing,Grenadine,Mangoes,Veal - Osso Bucco,Flour - Whole Wheat
2,2,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Juice - V8 Splash,"Veal - Brisket, Provimi,bnls",Anchovy Paste - 56 G Tube
3,3,Olives - Stuffed,Wine - Charddonnay Errazuriz,Assorted Desserts,Soup - Campbells Tomato Ravioli,Juice - Orange
4,4,"Tart Shells - Sweet, 4",Artichokes - Jerusalem,Sprouts - Baby Pea Tendrils,Chocolate - Dark,"Appetizer - Mini Egg Roll, Shrimp"
...,...,...,...,...,...,...
995,995,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
996,996,"Mushroom - Porcini, Dry",Bandage - Flexible Neon,Sugar - Fine,Table Cloth 62x114 White,Hot Chocolate - Individual
997,997,Macaroons - Two Bite Choc,Apricots - Halves,"Potatoes - Instant, Mashed",Arizona - Green Tea,"Soup - Campbells, Lentil"
998,998,"Veal - Inside, Choice",Bread Fig And Almond,Campari,Fondant - Icing,"Salmon - Atlantic, Skin On"


In [109]:
# metrica "city block"
sim_matrix = pd.DataFrame(1/(1 + squareform(pdist(pivot, metric='cityblock'))),index=pivot.index,columns=pivot.index)


make_reco(sim_matrix)

,CustomerID,Rec.Product1,Rec.Product2,Rec.Product3,Rec.Product4,Rec.Product5
0,0,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Sausage - Liver,Knife Plastic - White,Bay Leaf
1,1,Tia Maria,"Sole - Dover, Whole, Fresh",Veal - Sweetbread,Foam Dinner Plate,Juice - Lime
2,2,Soup - Campbells Bean Medley,Beef Wellington,Wine - Ej Gallo Sierra Valley,Butter - Unsalted,Knife Plastic - White
3,3,Beef - Texas Style Burger,Wine - Chardonnay South,"Wine - White, Colubia Cresh",Squid - Tubes / Tenticles 10/20,"Cheese - Boursin, Garlic / Herbs"
4,4,Veal - Sweetbread,Knife Plastic - White,Beef - Montreal Smoked Brisket,Quiche Assorted,"Salsify, Organic"
...,...,...,...,...,...,...
995,995,Curry Paste - Madras,Fond - Neutral,Wine - Ej Gallo Sierra Valley,"Cheese - Boursin, Garlic / Herbs",Knife Plastic - White
996,996,Yoghurt Tubes,Sausage - Breakfast,Wine - Blue Nun Qualitatswein,Muffin - Carrot Individual Wrap,Knife Plastic - White
997,997,Vanilla Beans,Sobe - Tropical Energy,Ice Cream Bar - Hageen Daz To,Soup - Campbells Tomato Ravioli,Lime Cordial - Roses
998,998,"Pepper - Paprika, Hungarian","Pepper - Black, Whole",Halibut - Steaks,Cod - Black Whole Fillet,Pork - Hock And Feet Attached


In [110]:
# metrica "cosine"
sim_matrix = pd.DataFrame(1/(1 + squareform(pdist(pivot, metric='cosine'))),index=pivot.index,columns=pivot.index)


make_reco(sim_matrix)

,CustomerID,Rec.Product1,Rec.Product2,Rec.Product3,Rec.Product4,Rec.Product5
0,0,Wine - Two Oceans Cabernet,Jagermeister,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,1,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Pork - Kidney,Bread - French Baquette
2,2,Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake
3,3,Bagel - Plain,Juice - Orange,Veal - Sweetbread,Olives - Stuffed,"Soup - Campbells, Beef Barley"
4,4,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted
...,...,...,...,...,...,...
995,995,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
996,996,"Salsify, Organic","Hickory Smoke, Liquid",Oil - Shortening - All - Purpose,Dc Hikiage Hira Huba,Wine - Alsace Gewurztraminer
997,997,Bread - Calabrese Baguette,French Pastry - Mini Chocolate,Foam Dinner Plate,Cheese - Parmesan Grated,Cheese - Mix
998,998,Flavouring - Orange,Longos - Chicken Wings,"Cheese - Brie, Triple Creme",Tia Maria,Fondant - Icing


In [111]:
# metrica "jaccard"
sim_matrix = pd.DataFrame(1/(1 + squareform(pdist(pivot, metric='jaccard'))),index=pivot.index,columns=pivot.index)


make_reco(sim_matrix)

,CustomerID,Rec.Product1,Rec.Product2,Rec.Product3,Rec.Product4,Rec.Product5
0,0,"Cheese - Brie, Triple Creme",Ecolab - Mikroklene 4/4 L,Flavouring - Orange,V8 - Berry Blend,Appetizer - Sausage Rolls
1,1,Sea Bass - Whole,Cheese - Wine,Garbag Bags - Black,Mayonnaise - Individual Pkg,Bread - Raisin Walnut Oval
2,2,Snapple - Iced Tea Peach,"Juice - Cranberry, 341 Ml",Rosemary - Dry,"Wine - Red, Colio Cabernet",Sponge Cake Mix - Chocolate
3,3,Wine - Hardys Bankside Shiraz,Table Cloth 81x81 White,Potatoes - Idaho 100 Count,"Wine - Cahors Ac 2000, Clos",Soup - Campbells Bean Medley
4,4,Muffin Batt - Blueberry Passion,Pie Filling - Cherry,Cocoa Butter,Wine - Blue Nun Qualitatswein,French Pastry - Mini Chocolate
...,...,...,...,...,...,...
995,995,Juice - Happy Planet,Lettuce - Frisee,Squid U5 - Thailand,Olive - Spread Tapenade,Eggplant - Asian
996,996,Wine - Redchard Merritt,Towels - Paper / Kraft,Hot Chocolate - Individual,Berry Brulee,"Lamb - Pieces, Diced"
997,997,Scampi Tail,Lettuce - Treviso,Mustard Prepared,Hot Chocolate - Individual,Zucchini - Yellow
998,998,"Coconut - Shredded, Sweet","Wine - Cahors Ac 2000, Clos","Cup - 6oz, Foam",Rice - Jasmine Sented,"Soup - Canadian Pea, Dry Mix"
